# Feature Profiling by Table (Pandas Version)

## Overview
Comprehensive feature profiling using **pandas** with memory-efficient chunk processing.

### Pandas Optimizations:
- **Chunked reading**: Process data in manageable chunks
- **Streaming statistics**: Calculate stats without loading full table
- **Memory efficient**: Use pandas iterators and explicit cleanup

### Statistics Calculated:
- Data type, % zeros, n_unique
- Most frequent value and percentage
- Percentiles: min, 1%, 50%, 99%, max, mean

### Outputs:
- Feature profiling CSVs per table with **separate statistics for In-Time vs OOT**
  - Each feature has two rows: one for 'In-Time' period, one for 'OOT' period
  - Includes `time_period` column to distinguish periods
- Individual boxplots for each feature (one PNG per feature)
  - Saved in table-specific folders: `plots/{table_name}/`
  - Each file named: `{feature_name}.png`
  - Comparing OOT vs in-time distributions

---


In [ ]:
%pip install --upgrade pandas==2 -i https://repo.td.com/repository/pypi-all/simple

In [ ]:
dbutils.library.restartPython()

In [ ]:
import pandas
print(pandas.__version__)

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from io import BytesIO
import gc
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')

# Helper functions
def save_pandas_to_csv_adls(df_pandas, adls_path):
    csv_string = df_pandas.to_csv(index=False)
    dbutils.fs.put(adls_path, csv_string, overwrite=True)
    print(f"✓ Saved CSV to {adls_path}")

def save_plot_to_adls(fig, adls_path, dpi=150):
    import tempfile, os
    buf = BytesIO()
    fig.savefig(buf, format='png', dpi=dpi, bbox_inches='tight')
    buf.seek(0)
    with tempfile.NamedTemporaryFile(mode='wb', suffix='.png', delete=False) as tmp:
        tmp.write(buf.getvalue())
        tmp_path = tmp.name
    dbutils.fs.cp(f"file:{tmp_path}", adls_path)
    os.remove(tmp_path)
    print(f"✓ Saved plot to {adls_path}")

# Optimized feature profiling function
def profile_feature_optimized(feature, period_df, is_cat):
    """Vectorized feature profiling - much faster than row-by-row"""
    stats = {
        'feature': feature,
        'data_type': 'categorical' if is_cat else 'numerical',
    }
    
    # Vectorized operations
    feature_data = period_df[feature]
    feature_clean = feature_data.dropna()
    
    if len(feature_clean) == 0:
        stats.update({
            'pct_zero': None, 'n_unique': 0, 'most_frequent_value': None,
            'pct_most_frequent': None, 'min': None, 'max': None,
            'p1': None, 'median': None, 'p99': None, 'mean': None
        })
        return stats
    
    stats['pct_zero'] = (feature_data == 0).mean()
    stats['n_unique'] = feature_clean.nunique()
    
    # Most frequent (mode)
    value_counts = feature_clean.value_counts()
    if len(value_counts) > 0:
        stats['most_frequent_value'] = value_counts.index[0]
        stats['pct_most_frequent'] = value_counts.iloc[0] / len(feature_clean)
    else:
        stats['most_frequent_value'] = None
        stats['pct_most_frequent'] = None
    
    if not is_cat:
        # Convert to numeric for numerical features
        feature_numeric = pd.to_numeric(feature_clean, errors='coerce').dropna()
        if len(feature_numeric) > 0:
            # Vectorized percentile calculations
            percentiles = np.percentile(feature_numeric, [0, 1, 50, 99, 100])
            stats.update({
                'min': float(percentiles[0]),
                'p1': float(percentiles[1]),
                'median': float(percentiles[2]),
                'p99': float(percentiles[3]),
                'max': float(percentiles[4]),
                'mean': float(feature_numeric.mean())
            })
        else:
            stats.update({
                'min': None, 'p1': None, 'median': None,
                'p99': None, 'max': None, 'mean': None
            })
    else:
        # Categorical: only min/max
        try:
            min_val = feature_clean.min()
            max_val = feature_clean.max()
            if hasattr(min_val, '__float__'):
                min_val = float(min_val)
            if hasattr(max_val, '__float__'):
                max_val = float(max_val)
            stats.update({
                'min': min_val, 'max': max_val,
                'p1': None, 'median': None, 'p99': None, 'mean': None
            })
        except:
            stats.update({
                'min': str(feature_clean.min()) if len(feature_clean) > 0 else None,
                'max': str(feature_clean.max()) if len(feature_clean) > 0 else None,
                'p1': None, 'median': None, 'p99': None, 'mean': None
            })
    
    return stats

# Optimized boxplot creation function
def create_boxplot_optimized(feature, intime_data, oot_data, table_folder_name, table_plot_folder):
    """Create boxplot for a single feature - optimized for parallel processing"""
    try:
        # Convert to numeric and numpy arrays (fast)
        intime_arr = pd.to_numeric(intime_data, errors='coerce').dropna()
        oot_arr = pd.to_numeric(oot_data, errors='coerce').dropna()
        
        intime_arr = np.array(intime_arr, dtype=np.float64)
        oot_arr = np.array(oot_arr, dtype=np.float64)
        
        intime_arr = intime_arr[np.isfinite(intime_arr)]
        oot_arr = oot_arr[np.isfinite(oot_arr)]
        
        if len(intime_arr) > 0 and len(oot_arr) > 0:
            fig, ax = plt.subplots(figsize=(10, 6))
            
            # HORIZONTAL boxplot
            bp = ax.boxplot([intime_arr, oot_arr], 
                           labels=['In-Time', 'OOT'], 
                           vert=False,  # HORIZONTAL
                           patch_artist=True,
                           showmeans=False, 
                           showfliers=True)
            
            # Style boxes
            colors = ['lightblue', 'lightcoral']
            for patch, color in zip(bp['boxes'], colors):
                patch.set_facecolor(color)
                patch.set_alpha(0.7)
            
            # Style outliers with LOW ALPHA for dense outliers
            for flier in bp['fliers']:
                flier.set_marker('o')
                flier.set_markerfacecolor('black')
                flier.set_markersize(3)
                flier.set_alpha(0.1)  # Very low alpha
            
            ax.set_title(f'{feature}\n({table_folder_name})', fontsize=12, fontweight='bold')
            ax.set_xlabel('Value', fontsize=10)
            ax.set_ylabel('Time Period', fontsize=10)
            ax.grid(True, alpha=0.3, axis='x', linestyle='--')
            
            plt.tight_layout()
            
            # Save
            plot_file = f"{table_plot_folder}{feature}.png"
            save_plot_to_adls(fig, plot_file, dpi=150)
            plt.close(fig)
            
            return True
        else:
            return False
    except Exception as e:
        return False

print("✓ Setup complete with optimizations")


In [ ]:
# Configuration
DATA_PATH = "abfss://home@edaaaazepcalayelaye0001.dfs.core.windows.net/MD_Artifacts/money-out/data/"
OUTPUT_PATH = "abfss://home@edaaaazepcalayelaye0001.dfs.core.windows.net/MD_Artifacts/money-out/mv/eda_validation/feature_profiling/"
PLOT_PATH = OUTPUT_PATH + "plots/"
dbutils.fs.mkdirs(OUTPUT_PATH)
dbutils.fs.mkdirs(PLOT_PATH)

SAMPLING_RATIO = 0.01
PLOT_SAMPLING_RATIO = 0.01
OOT_START_DATE = '2024-01-01'

# Feature tables to analyze
TABLES = [
    ("cust", "cust_basic_sumary", ''),
    ("cust", "batch_credit_bureau", ''),
    ("dem", "acct", 2438),
    ("cc", "acct", 2444),
    ("loc", "acct", 2442),
    ("loan", "acct", 2439),
    ("mtg", "acct", 2440),
    ("inv", "acct", 1331),
    ("dem", "acct_trans", 2438),
    ("cc", "acct_trans", 2444),
]

# Load metadata
feature_metadata_rows = spark.read.text(f"{DATA_PATH}/feature/feature_metadata.jsonl").collect()
feature_metadata = json.loads('\n'.join([row.value for row in feature_metadata_rows]))

print("✓ Config loaded")


## Processing Strategy: Sampled Full-Table (Accuracy Prioritized)

### Why This Approach?
This notebook calculates **median, percentiles (p1, p99)** which **CANNOT be calculated incrementally**. We must see all values to sort/rank them accurately.

### Memory Efficiency:
- **Memory usage**: Scales with SAMPLING_RATIO
- **Mitigation**: Process one table at a time (10 tables total), free memory between tables
- **Recommendation for memory issue**: Use `SAMPLING_RATIO = 0.01` (1%) for accurate results with manageable memory

### How It Works:
```
For each table (10 total):
  1. Load FULL table via Spark (efficient Parquet reading)
  2. Apply sampling at Spark level: .sample(fraction=SAMPLING_RATIO)
  3. Convert to pandas: .toPandas()
  4. Calculate accurate statistics:
     - median: df[col].median() ← requires sorted values
     - p99: df[col].quantile(0.99) ← requires percentile calculation
     - mean, min, max, n_unique, etc.
  5. Free memory before next table (del df; gc.collect())
```

### Why Incremental Doesn't Work Here:
- ❌ median(chunk1) + median(chunk2) ≠ median(all_data)
- ❌ p99(chunk1) combined with p99(chunk2) ≠ p99(all_data)
- ✅ Must see all sampled values together to calculate correct percentiles
- ✅ 1% sampling gives exact statistics on representative sample

### Alternative Considered:
Could use approximate algorithms (T-Digest, Q-Digest) for streaming percentiles, but:
- ❌ Introduces approximation error
- ❌ Complex to implement and debug
- ✅ 1% sampling gives exact results with manageable memory
- ✅ Simpler code is easier to maintain

---


## Success Criteria and Expected Results

### ✅ **Profiling Succeeds If**:
- All tables processed successfully
- Statistics calculated for **all features** (numerical + categorical) in metadata
- **Separate statistics** calculated for In-Time vs OOT periods
- No excessive missing values (>99.9%) unless expected
- Reasonable value ranges (no extreme outliers unless business-valid)
- Categorical features have reasonable cardinality
- **Time-period comparison** shows expected differences between In-Time and OOT

### 📊 **Statistics Calculated Per Feature (Per Time Period)**:
| Statistic | Numerical | Categorical | Notes |
|-----------|-----------|-------------|-------|
| time_period | ✓ | ✓ | 'In-Time' or 'OOT' |
| feature | ✓ | ✓ | Feature name |
| data_type | ✓ | ✓ | Identifies feature type |
| pct_zero | ✓ | ✓ | % of values that are 0 |
| n_unique | ✓ | ✓ | Number of distinct values |
| most_frequent_value | ✓ | ✓ | Mode |
| pct_most_frequent | ✓ | ✓ | % of samples with mode |
| min | ✓ | ✓ | Minimum value |
| max | ✓ | ✓ | Maximum value |
| p1 | ✓ | ✗ | 1st percentile |
| median (p50) | ✓ | ✗ | 50th percentile |
| p99 | ✓ | ✗ | 99th percentile |
| mean | ✓ | ✗ | Average value |

### 📈 **Time-Period Analysis**:
Each feature has **two rows** in the output CSV:
- **Row 1**: Statistics for 'In-Time' period (before 2024-01-01)
- **Row 2**: Statistics for 'OOT' period (2024-01-01 and after)

**What to Compare**:
- **Distributions**: Compare median, mean, percentiles between periods
- **Sparsity**: Compare pct_zero (may increase/decrease over time)
- **Cardinality**: Compare n_unique for categorical features
- **Ranges**: Compare min/max values (may indicate data quality issues)
- **Mode**: Compare most_frequent_value (distribution shifts)

### ⚠️ **Potential Issues to Flag**:
- **Features with >99% zeros** (may be redundant or sparse)
- **Features with only 1 unique value** (constant features - no information)
- **Features with extreme ranges** (may need normalization or clipping)
- **Categorical features with very high cardinality** (>1000 categories - may need bucketing)
- **Features missing from certain tables** (expected for table-specific features)
- **Large differences between In-Time and OOT**:
  - Significant shifts in median/mean (potential drift)
  - Large changes in pct_zero (sparsity changes)
  - Cardinality changes in categoricals (new categories appear/disappear)

### 📊 **Boxplot Visualizations**:
- **One boxplot per numerical feature** comparing In-Time vs OOT
- Saved individually: `plots/{table_name}/{feature_name}.png`
- **What to look for**:
  - Distribution shifts (boxes at different positions)
  - Spread changes (box width differences)
  - Outlier patterns (fliers in different locations)
  - Median differences (horizontal line position)

---


In [ ]:
print("="*80)
print("FEATURE PROFILING")
print("="*80)

for fam_name, table, fam in TABLES:
    print(f"\nProcessing: {fam_name}-{table}")
    
    table_path = f"{DATA_PATH}/feature/{table}/parquet" if not fam else f"{DATA_PATH}/feature/{table}_{fam}/parquet"
    table_meta_key = table if not fam else f"{table}_{fam}"
    
    if fam_name not in feature_metadata or table_meta_key not in feature_metadata[fam_name]:
        continue
    
    num_features = feature_metadata[fam_name][table_meta_key].get("num_features", [])
    cat_features = list(feature_metadata[fam_name][table_meta_key].get("cat_features", {}).keys())
    
    # OPTIMIZATION: Load table once, process features in batches to avoid driver memory limit
    df_spark = spark.read.format("parquet").load(table_path)
    if SAMPLING_RATIO < 1.0:
        df_spark = df_spark.sample(fraction=SAMPLING_RATIO, withReplacement=False, seed=42)
    
    # IMPORTANT FIX: Process features in batches to avoid exceeding driver result size limit
    # Instead of loading all features at once (which can exceed 126 GB limit),
    # we load features in small batches
    
    BATCH_SIZE = 4  # Process 4 features at a time to avoid driver memory limit
    all_features = num_features + cat_features
    all_stats = []
    
    # Always include efectv_dt if available
    has_efectv_dt = 'efectv_dt' in df_spark.columns
    
    # Process features in batches
    for batch_idx in range(0, len(all_features), BATCH_SIZE):
        batch_features = all_features[batch_idx:batch_idx + BATCH_SIZE]
        batch_features_in_table = [f for f in batch_features if f in df_spark.columns]
        
        if len(batch_features_in_table) == 0:
            continue
        
        # Load only this batch of features
        cols_batch = batch_features_in_table
        if has_efectv_dt:
            cols_batch = ['efectv_dt'] + cols_batch
        
        df_batch = df_spark.select(cols_batch).toPandas()
        
        # Split by time period if efectv_dt is available
        if has_efectv_dt:
            df_batch['efectv_dt'] = pd.to_datetime(df_batch['efectv_dt'])
            oot_date = pd.to_datetime(OOT_START_DATE)
            df_batch['time_period'] = np.where(df_batch['efectv_dt'] >= oot_date, 'OOT', 'In-Time')
            
            # Profile this batch for both time periods
            for period_name in ['In-Time', 'OOT']:
                period_df = df_batch[df_batch['time_period'] == period_name]
                
                if len(period_df) > 0:
                    for feature in batch_features_in_table:
                        if feature not in period_df.columns:
                            continue
                        is_cat = feature in cat_features
                        
                        # Use optimized profiling function
                        feature_stats = profile_feature_optimized(feature, period_df, is_cat)
                        feature_stats['time_period'] = period_name
                        all_stats.append(feature_stats)
        else:
            # No time splitting - profile all data for this batch
            for feature in batch_features_in_table:
                if feature not in df_batch.columns:
                    continue
                is_cat = feature in cat_features
                
                feature_stats = profile_feature_optimized(feature, df_batch, is_cat)
                feature_stats['time_period'] = 'All'
                all_stats.append(feature_stats)
        
        # Free memory after each batch
        del df_batch
        gc.collect()
        
        # Reduce print frequency to avoid I/O stream timeout
        if batch_idx > 0 and (batch_idx % 50 == 0):
            print(f"    Processed {min(batch_idx, len(all_features))}/{len(all_features)} features...")
    
    # Save profiling results
    if all_stats:
        results_df = pd.DataFrame(all_stats)
        col_order = ['time_period', 'feature', 'data_type'] + [c for c in results_df.columns if c not in ['time_period', 'feature', 'data_type']]
        results_df = results_df[col_order]
        save_pandas_to_csv_adls(results_df, f"{OUTPUT_PATH}feature_profile_{fam_name}_{table}.csv")
    
    # Create boxplots (need to reload data with time_period for plotting)
    if has_efectv_dt:
        # Load only numerical features needed for plotting in batches
        plot_features = [f for f in num_features if f in df_spark.columns]
        
        if len(plot_features) > 0:
            print(f"  Creating {len(plot_features)} individual boxplots (parallel processing)...")
            
            table_folder_name = f"{fam_name}_{table}" if not fam else f"{fam_name}_{table}_{fam}"
            table_plot_folder = f"{PLOT_PATH}{table_folder_name}/"
            dbutils.fs.mkdirs(table_plot_folder)
            
            saved_count = 0
            failed_count = 0
            
            # Process plotting in feature batches to avoid memory AND I/O stream issues
            PLOT_BATCH_SIZE = 4  # Smaller batches to avoid overwhelming I/O
            
            for plot_batch_idx in range(0, len(plot_features), PLOT_BATCH_SIZE):
                plot_batch = plot_features[plot_batch_idx:plot_batch_idx + PLOT_BATCH_SIZE]
                
                # Load only this batch for plotting
                df_plot = df_spark.select(['efectv_dt'] + plot_batch).toPandas()
                df_plot['efectv_dt'] = pd.to_datetime(df_plot['efectv_dt'])
                oot_date = pd.to_datetime(OOT_START_DATE)
                df_plot['time_period'] = np.where(df_plot['efectv_dt'] >= oot_date, 'OOT', 'In-Time')
                
                # Pre-filter masks
                intime_mask = df_plot['time_period'] == 'In-Time'
                oot_mask = df_plot['time_period'] == 'OOT'
                
                # OPTIMIZATION: Reduce parallel jobs to avoid I/O stream timeout
                # Use n_jobs=2 instead of 4, and verbose=0 to suppress output
                results = Parallel(n_jobs=2, backend='threading', verbose=0)(
                    delayed(create_boxplot_optimized)(
                        feature,
                        df_plot.loc[intime_mask, feature],
                        df_plot.loc[oot_mask, feature],
                        table_folder_name,
                        table_plot_folder
                    )
                    for feature in plot_batch
                )
                
                saved_count += sum(results)
                failed_count += len(results) - sum(results)
                
                del df_plot
                gc.collect()
                
                # Print progress less frequently to avoid I/O congestion
                if plot_batch_idx > 0 and (plot_batch_idx % 25 == 0):
                    print(f"      Boxplots: {saved_count + failed_count}/{len(plot_features)} processed...")
            
            print(f"  ✓ Boxplots saved: {saved_count} successful, {failed_count} failed")
            print(f"    Location: {table_plot_folder}")
    
    del df_spark
    gc.collect()

print("\n✓ Feature profiling complete")
